In [218]:
import pandas as pd

# 4번째(인덱스 3) + 5번째(인덱스 4) 행을 멀티헤더로 불러오기
df_raw = pd.read_excel("school_info_0306.xlsx", header=[3, 4])

# df_raw.columns

# 컬럼 공백, 개행 제거 및 멀티컬럼 평탄화
def clean_columns(df): 
    df.columns = [
        f"{a.strip().replace('\n', '').replace(' ', '')}_{b.strip().replace('\n', '').replace(' ', '')}"
        if 'Unnamed' not in str(b) else a.strip().replace('\n', '')
        for a, b in df.columns
    ]
    if df.columns[0].startswith("Unnamed"):
        df.drop(columns=[df.columns[0]], inplace=True)
    return df.reset_index(drop=True)
df_raw = clean_columns(df_raw)
df = df_raw.reset_index(drop=True)
df.head()

,프로그램구분,기관,일련번호,지역,국가명,대학명(국문),대학명(영문),지원자격_최소학점,지원자격_어학성적,지원자격_특이사항,참고사항,수학가능학과/영어강의목록 등,Factsheet 여부,교환학생수기 여부,유의사항,웹사이트
0,일반교환,자매대학,E0009,북미,캐나다,"퀘백대, 몬트리올","Université du Québec, Montréal",NaN,D-1,프랑스어 필수,"* 위치: Montreal, Quebec, Canada\n* 특징: Quebec 州...","★ 수학가능학과: Arts, Communication, Education, Mana...",Y(2022),x,★ 2학기만 파견 가능,https://uqam.ca/
1,일반교환,자매대학,E0010,유럽,그리스,아테네경제경영대,Athens University of Economics and Business,NaN,IELTS 6.0\nTOEFL (iBT) 80,TOEIC/TOEFL ITP 제외,"* 위치: Athens\n* 특징: 학생 수 약 3,000명\n #...",★ 수학가능학과: International & European Economic St...,Y(2024),Y(2018),NaN,https://www.aueb.gr/en/content/international-r...
2,일반교환,자매대학,E0013,유럽,네덜란드,HZ대,HZ University of Applied Sciences,NaN,TOEFL(iBT) 80\nIELTS 6.0\nTOEIC 960,"TOEFL ITP 제외\n홈에디션 제외\n*TOEIC Listening, Readi...","* 위치: Vlissingen\n* 특징: 학생 수 약 4,800명 \n ...","★ 수학가능학과: Chemistry, Civil Engineering, Engine...",Y(2024),X,NaN,https://hz.nl/en/exchange-programmes
3,일반교환,자매대학,E0014,유럽,네덜란드,라드바우드대,Radboud University,법학과 수업을 \n36학점 이상 취득,"TOEFL iBT 80\nIELTS 6.0\nTOEIC 1,200 이상","*TOEIC Listening, Reading 865 이상\nTOEIC Speaki...",* 위치: Nijmegen\n* 특징: 연구 중심 공립 대학\n 학...,★ 수학가능학과: Law\n\n★ 수강가능과목(2024): https://www.r...,Y(2024-2025),Y(2016-2019),NaN,1) https://www.ru.nl/en/education/more-educati...
4,일반교환,자매대학,E0015,유럽,네덜란드,삭시온대,Saxion University of Applied Sciences,NaN,TOEFL(iBT) 80\nIELTS 6.0\nTOEIC 960,* 토플 홈에디션 인정\n* 토플 마이베스트스코어 인정 불가\n*TOEIC List...,"* 위치: Enschede, Deventer, Apeldoom \n* 특징: 연구 ...",★ 수학가능학과: Creative Media and Game Technologies...,Y(2021),Y(2014-2019),NaN,https://www.saxion.edu/programmes/exchange-pro...


In [219]:
# [전처리] 마지막 행 제거
# 이유: 엑셀 원본 마지막 행은 "합계" 같은 통계/요약 행으로, 실제 데이터가 아님
# df.shape[0]은 전체 행 수, df.index[-1]은 마지막 행 인덱스
df = df.drop(df.index[-1]).reset_index(drop=True)

df['국가명'] = df['국가명'].str.strip() # 국가명 공백 제거

# [전처리] 전체 데이터에서 \n 제거
# 문자열 컬럼에 대해서만 \n을 공백 없이 제거 (교환학생\n+인턴십 → 교환학생+인턴십)
# df = df.applymap(lambda x: x.replace('\n', '') if isinstance(x, str) else x)

# [전처리] '대학명(국문)' 컬럼에서 ★, ● 제거
df['대학명(국문)'] = df['대학명(국문)'].str.replace('★', '', regex=False)
df['대학명(국문)'] = df['대학명(국문)'].str.replace('●', '', regex=False)
# 앞뒤 공백도 함께 정리
df['대학명(국문)'] = df['대학명(국문)'].str.strip()
# [전처리] '대학명(영문)' 컬럼의 앞뒤 공백 제거 (중간 공백은 유지)
df['대학명(영문)'] = df['대학명(영문)'].str.strip()

# [전처리] '유의사항' 컬럼에서 ★, ● 제거
df['유의사항'] = df['유의사항'].str.replace('★ ', '', regex=False)


# [전처리] '웹사이트' 컬럼에서 1) 2) 제거
def clean_website_links(text):
    if pd.isna(text):
        return text
    # 링크만 추출
    links = re.findall(r'https?://[^\s\)]+', text)
    # 쉼표로 join 후 반환
    return ', '.join(links)

# 웹사이트 컬럼 덮어쓰기
df['웹사이트'] = df['웹사이트'].apply(clean_website_links)


In [220]:
print(f"전체 데이터 수 : {df.shape[0]}") # 컬럼행 제외, 실제 데이터 수
print("=" * 60)
for col in df.columns:
    print(f"[{col}]")
    print(f"고유값 수: {df[col].nunique()}")
    # print(f"고유값 리스트: {df[col].unique()[:10]}")  # 너무 많으면 10개만
    print(f"고유값 리스트: {df[col].unique()}") # 전체 표시
    print("-" * 60)

전체 데이터 수 : 338
[프로그램구분]
고유값 수: 5
고유값 리스트: ['일반교환' '방문교환' '인턴십 ' '특별' '교환학생\n+인턴십']
------------------------------------------------------------
[기관]
고유값 수: 4
고유값 리스트: ['자매대학' 'ACUCA' 'BCI' 'SAF']
------------------------------------------------------------
[일련번호]
고유값 수: 338
고유값 리스트: ['E0009' 'E0010' 'E0013' 'E0014' 'E0015' 'E0016' 'E0017' 'E0018' 'E0026'
 'E0027' 'E0031' 'E0034' 'E0036' 'E0037' 'E0038' 'E0039' 'E0040' 'E0041'
 'E0042' 'E0043' 'E0044' 'E0045' 'E0046' 'E0047' 'E0048' 'E0049' 'E0050'
 'E0051' 'E0052' 'E0054' 'E0055' 'E0057' 'E0058' 'E0060' 'E0061' 'E0062'
 'E0063' 'E0066' 'E0067' 'E0068' 'E0070' 'E0071' 'E0072' 'E0073' 'E0077'
 'E0079' 'E0080' 'E0081' 'E0082' 'E0083' 'E0084' 'E0085' 'E0086' 'E0087'
 'E0089' 'E0091' 'E0092' 'E0093' 'E0094' 'E0095' 'E0096' 'E0097' 'E0098'
 'E0099' 'E0100' 'E0101' 'E0102' 'E0103' 'E0104' 'E0105' 'E0106' 'E0107'
 'E0108' 'E0109' 'E0110' 'E0113' 'E0114' 'E0115' 'E0116' 'E0117' 'E0118'
 'E0119' 'E0120' 'E0122' 'E0123' 'E0124' 'E0126' 'E0127' 'E

In [221]:
# df['일련번호'].isna().sum()
# df['일련번호'].value_counts()[df['일련번호'].value_counts() > 1]
# # 일련번호 공백 및 개행 제거
# df['일련번호'] = df['일련번호'].astype(str).str.strip().str.replace('\n', '', regex=False)

# # 다시 중복 확인
# duplicates = df[df['일련번호'].duplicated(keep=False)]

# # 확인
# print(duplicates[['일련번호']])
# (df['일련번호'] == 'nan').sum()


# print("정제 후 고유값 수:", df['일련번호'].nunique())
# print("전체 행 수:", df.shape[0])

# print("제거 후 전체 행 수:", df.shape[0])
# print("제거 후 마지막 일련번호:", df['일련번호'].iloc[-1])

In [222]:
# # 한글이 하나도 없는 값만 필터링
# import re

# def is_english_only(text):
#     if pd.isna(text):
#         return False
#     return not re.search(r'[가-힣]', str(text))

# # 영어로 된 대학명만 추출
# eng_named_univs = df[df['대학명(국문)'].apply(is_english_only)]
# eng_named_univs


In [223]:
# country_to_continent = {
#     # 북미
#     '캐나다': '북미', '미국': '북미', '멕시코': '북미', '과테말라': '북미', '도미니카공화국': '북미',

#     # 남미
#     '브라질': '남미', '아르헨티나': '남미', '에콰도르': '남미', '우루과이': '남미',
#     '칠레': '남미', '콜롬비아': '남미', '페루': '남미',

#     # 유럽
#     '그리스': '유럽', '네덜란드': '유럽', '덴마크': '유럽', '독일': '유럽', '라트비아': '유럽',
#     '러시아': '유럽', '루마니아': '유럽', '리투아니아': '유럽', '벨기에': '유럽',
#     '스웨덴': '유럽', '스위스': '유럽', '스페인': '유럽', '아일랜드': '유럽', '영국': '유럽',
#     '에스토니아': '유럽', '오스트리아': '유럽', '우크라이나': '유럽', '이탈리아': '유럽',
#     '체코': '유럽', '크로아티아': '유럽', '포르투갈': '유럽', '폴란드': '유럽',
#     '프랑스': '유럽', '핀란드': '유럽', '헝가리': '유럽', '아제르바이잔': '유럽',

#     # 아시아
#     '대만': '아시아', '말레이시아': '아시아', '베트남': '아시아', '몽골': '아시아',
#     '우즈베키스탄': '아시아', '인도네시아': '아시아', '싱가포르': '아시아', '일본': '아시아',
#     '중국': '아시아', '방글라데시': '아시아', '카자흐스탄': '아시아', '태국': '아시아',
#     '홍콩': '아시아', '인도': '아시아', '미얀마': '아시아', '키르기스스탄': '아시아',
#     '튀르키예(터키)': '아시아', '필리핀': '아시아', '미얀마': '아시아',

#     # 오세아니아
#     '호주': '오세아니아', '뉴질랜드': '오세아니아',

#     # 아프리카
#     '튀니지': '아프리카',
# }

In [224]:
import pandas as pd
import numpy as np
import re

def format_gpa(value):
    if pd.isna(value):
        return '-'

    value = str(value).strip()

    # 1. '3.0/4.5' 형태 → 그대로 + ' 이상'
    if re.fullmatch(r'\d+(\.\d+)?/4\.5', value):
        return f"{value} 이상"

    # 2. 'X.XX 이상' 형태 → 'X.XX/4.5 이상'
    match = re.fullmatch(r'(\d+(\.\d+)?) 이상', value)
    if match:
        return f"{match.group(1)}/4.5 이상"

    # 3. '학점 X.XX 이상' 형태 → 'X.XX/4.5 이상'
    match = re.fullmatch(r'학점\s*(\d+(\.\d+)?) 이상', value)
    if match:
        return f"{match.group(1)}/4.5 이상"

    # 4. 숫자만 있는 경우 (예: 2.81) → '2.81/4.5 이상'
    if re.fullmatch(r'\d+(\.\d+)?', value):
        return f"{value}/4.5 이상"

    # 5. 나머지 (문자 그대로 유지)
    return value

# 적용
df['지원자격_최소학점'] = df['지원자격_최소학점'].apply(format_gpa)

In [225]:
df['지원자격_최소학점'].unique()

array(['-', '법학과 수업을 \n36학점 이상 취득', '2.25/4.5 이상', '3.0/4.5 이상',
       '2.81/4.5 이상', '3.15/4.5 이상', '3.7/4.5 이상', '2.5/4.5 이상',
       'Jinan University(暨南大学)+I233', '3.04/4.5 이상', '(Hybrid Option A)',
       '(Hybrid Option B)'], dtype=object)

In [226]:
# 어학 성적 등급표 전처리

# 기준 등급표
grade_mapping = {
    '영어(A)': {
        'TOEFL': {'A-1': 85, 'A-2': 80, 'A-3': 75, 'A-4': 70, 'A-5': 60},
        'IELTS': {'A-1': 6.5, 'A-2': 6.0, 'A-3': 5.5, 'A-4': 5.0, 'A-5': 4.5},
        'TOEIC': {'A-1': 900, 'A-2': 850, 'A-3': 800, 'A-4': 750, 'A-5': 700},
        'TOEFL ITP': {'A-1': 600, 'A-2': 560, 'A-3': 545, 'A-4': 530, 'A-5': 515},
    },
    '중국어(B)': {
        '신HSK': {'B-1': 6, 'B-2': 5, 'B-3': 4},
    },
    '일본어(C)': {
        'JLPT': {'C-1': 'N1', 'C-2': 'N2'},
        'JPT': {'C-1': 900, 'C-2': 600},
    },
    '프랑스어(D)': {
        'DELF': {'D-1': 'B2', 'D-2': 'B1', 'D-3': 'A2'},
    },
    '독일어(E)': {
        'ZD': {'E-1': 'B2', 'E-2': 'B1', 'E-3': 'A2'},
    },
}

import re

# 언어권별 키워드 배열 정의
LANGUAGE_KEYWORDS = {
    '영어(A)': [
        'toefl', 'ibt', 'itp', 'ielts', 'toeic',
        'academic', 'duolingo', '듀오링고',
        '영어', '영어: toefl', '영어b2'
        'a-1', 'a-2', 'a-3', 'a-4', 'a-5',
        'a1', 'a2', 'a3', 'a4', 'a5'
    ],
    '프랑스어(D)': [
        '프랑스어', 'delf', 'dele', 
        'delf b2', 'deleb1', 'delec1', 'delf 50/100',
        'd-1', 'd-2', 'd-3',
        'd1', 'd2', 'd3'
    ],
    '중국어(B)': [
        '중국어', 'hsk', '신hsk',
        'b-3', 'b3',
        # 'b-1', 'b-2',  # 영어권에도 있는 등급
        # 'b1', 'b2',   # 영어권에도 있는 등급
    ],
    '일본어(C)': [
        'jlpt', 'jpt', '일본어', 
        'n1', 'n2',
        'c-1', 'c-2',
        'c1', 'c2'
    ],
    '독일어(E)': [
        '독일어', 'zd','독일어e2',
        'e-1', 'e-2', 'e-3', 
        'e1', 'e2', 'e3', 
    ],
    '기타': [
        'celpe', '포르투갈어', 'intermediario', 'superior'
    ]
}

def extract_language_zones_with_country(row):
    text = str(row['지원자격_어학성적']).lower()
    country = row['국가명']
    
    if pd.isna(text) or text.strip() == '-':
        return '기타'

    matched = set()

    for lang, keywords in LANGUAGE_KEYWORDS.items():
        for kw in keywords:
            if kw in text:
                matched.add(lang)

    # 조건: 중국 or 대만 & b-1, b-2 등급 포함 시, '중국어(B)' 추가
    if country in ['중국', '대만']:
        if re.search(r'\bb[- ]?[1-2]\b', text):  # b-1, b1, b 1 모두 포함
            matched.add('중국어(B)')

    return ', '.join(sorted(matched)) if matched else '기타'

# 적용
df['언어권구분'] = df.apply(extract_language_zones_with_country, axis=1)


df_pairs = df[['국가명', '언어권구분', '지원자격_어학성적']].apply(tuple, axis=1).unique()
df_pairs

# df[df['언어권구분'] == '기타'][['국가명', '지원자격_어학성적']]

array([('캐나다', '프랑스어(D)', 'D-1'),
       ('그리스', '영어(A)', 'IELTS 6.0\nTOEFL (iBT) 80'),
       ('네덜란드', '영어(A)', 'TOEFL(iBT) 80\nIELTS 6.0\nTOEIC 960'),
       ('네덜란드', '영어(A)', 'TOEFL iBT 80\nIELTS 6.0\nTOEIC 1,200 이상'),
       ('네덜란드', '영어(A)', '영어 B2'), ('네덜란드', '영어(A)', 'A-2'),
       ('덴마크', '영어(A)', 'TOEFL(iBT) 83\nIELTS 6.5'),
       ('독일', '영어(A)', '영어 B2'),
       ('독일', '독일어(E), 영어(A)', 'TOEFL(iBT) 80\nIELTS 6.0\nTOEIC 850\n독일어 E2'),
       ('독일', '독일어(E), 영어(A)', '영어B2 \n독일어E2'),
       ('독일', '독일어(E), 영어(A)', 'TOEFL(iBT) 78\n독일어 E1'),
       ('독일', '독일어(E), 영어(A)', '영어 B2\n독일어 E2'), ('라트비아', '기타', 'B2'),
       ('라트비아', '영어(A)', 'TOEFL(iBT) 75\nIELTS 5.5\nTOEIC 800'),
       ('러시아', '영어(A)', 'TOEFL 80\nIELTS 6.0'),
       ('러시아', '영어(A)', 'TOEFL(iBT) 75\nIELTS 5.5\nTOEIC 800'),
       ('러시아', '영어(A)', '영어 B2'), ('루마니아', '영어(A)', '영어 B2'),
       ('루마니아', '영어(A)', 'TOEFL(iBT) 80\nIELTS 6.0\nTOEIC 850'),
       ('리투아니아', '영어(A)', '영어 B2'),
       ('벨기에', '영어(A), 프랑스어(D)', '영어